In [7]:
!pip install neurokit2

In [8]:
!pip install hrv-analysis
!pip install pyhrv

In [29]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 48.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.1 MB/s eta 0:00:00


In [9]:
#import tensorflow as tf
#import seaborn as sns

In [10]:
#from scipy.interpolate import interp1d

In [11]:
import numpy as np 
import pandas as pd 

In [33]:
import torch
#import onnx
import onnxruntime as ort
from joblib import load

In [31]:
#import keras
import neurokit2 as nk
import pyhrv.tools as tools
import pyhrv.frequency_domain as fd
import matplotlib
pd.options.mode.chained_assignment = None
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [13]:
from scipy import signal

In [14]:
class ECG_RR():
    def __init__(self,info,ecg_signals_1,name):
        self.ECG = info
        self.ecg_signals_1 = ecg_signals_1
        self.name = name
    def RR(self,sp,RR_d):
        RR_d = RR_d
        if len(self.ECG[self.name]) > 3:
            for i in range(0,len(self.ECG[self.name])-1):
                RR_d = sp(RR_d,list(self.ECG[self.name])[i+1] - list(self.ECG[self.name])[i])
        else :RR_d =0
        return RR_d
    def R_m(self,sp):
        if len(self.ecg_signals_1[self.ecg_signals_1[self.name]>0].ECG_Raw):
            return sp(self.ecg_signals_1[self.ecg_signals_1[self.name]>0].ECG_Raw)
        else: return 0

In [15]:
def timedomain_pic(ecg_signals_1,info,name,results):
    ECG = ECG_RR(info,ecg_signals_1,"ECG_R_Peaks")
    name = str(name)+"_"
    results[name+'Max_R'] = [ECG.R_m(max)]
    results[name+'Min_R'] = [ECG.R_m(min)]
    results[name+'Max_RR'] = [ECG.RR(max,0)*(1/500)]
    results[name+'Min_RR'] = [ECG.RR(min,500)*(1/500)]
    ECG = ECG_RR(info,ecg_signals_1,"ECG_P_Peaks")
    results[name+'Max_P'] = [ECG.R_m(max)]
    results[name+'Min_P'] = [ECG.R_m(min)]
    ECG = ECG_RR(info,ecg_signals_1,"ECG_Q_Peaks")
    results[name+'Max_Q'] = [ECG.R_m(max)]
    results[name+'Min_Q'] = [ECG.R_m(min)]
    ECG = ECG_RR(info,ecg_signals_1,"ECG_S_Peaks")
    results[name+'Max_S'] = [ECG.R_m(max)]
    results[name+'Min_S'] = [ECG.R_m(min)]
    ECG = ECG_RR(info,ecg_signals_1,"ECG_T_Peaks")
    results[name+'Max_T'] = [ECG.R_m(max)]
    results[name+'Min_T'] = [ECG.R_m(min)]
    return results

In [16]:
def timedomain_HRV(ecg_signals_1,info,name,results):
    try:
        rr_intervals_list = info['ECG_R_Peaks']
        nni_2 = tools.nn_intervals(rr_intervals_list)
        result = fd.welch_psd(nni=nni_2,show=False, show_param=False,legend=False)
        matplotlib.pyplot.close(fig="all")
    except:
        return results
    name = str(name)+"_"
    results[name+'VLF_Peak (Hz)'] = [result[1][0]]
    results[name+'LF_Peak (Hz)'] = [result[1][1]]
    results[name+'HV_Peak (Hz)'] = [result[1][2]]
    results[name+'VLF_Abs (ms2)'] = [result[2][0]]
    results[name+'LF_Abs (ms2)'] = [result[2][1]]
    results[name+'HV_Abs (ms2)'] = [result[2][2]]
    results[name+'VLF_Rel (%)'] = [result[3][0]]
    results[name+'LF_Rel (%)'] = [result[3][1]]
    results[name+'HV_Rel (%)'] = [result[3][2]]
    results[name+'VLF_Log (-)'] = [result[4][0]]
    results[name+'LF_Log (-)'] = [result[4][1]]
    results[name+'HV_Log (-)'] = [result[4][2]]
    results[name+'LF_Norm (-)'] = [result[5][0]]
    results[name+'HV_Norm (-)'] = [result[5][1]]
    results[name+'LF/HF (-)'] = [result[6]]
    results[name+'Total Power (ms)'] = [result[7]]
    return results

In [17]:
def timedomain(ecg_signals_1,info,name,results):
    rr = info['ECG_R_Peaks'][:-1]
    name = str(name)+"_"
    rr_ecg = np.diff(rr)
    rr = rr_ecg*10
    hr = 60000/rr
    results[name+'Mean RR (ms)'] = [np.mean(rr)]
    results[name+'STD RR/SDNN (ms)'] = [np.std(rr)]
    results[name+'Mean HR (Kubios\' style) (beats/min)'] = [60000/np.mean(rr)]
    results[name+'Mean HR (beats/min)'] = [np.mean(hr)]
    results[name+'STD HR (beats/min)'] = [np.std(hr)]
    results[name+'Min HR (beats/min)'] = [np.min(hr)]
    results[name+'Max HR (beats/min)'] = [np.max(hr)]
    results[name+'RMSSD (ms)'] = [np.sqrt(np.mean(np.square(np.diff(rr))))]
    results[name+'NNxx'] = [np.sum(np.abs(np.diff(rr)) > 50)*1]
    results[name+'pNNxx (%)'] = [100 * np.sum((np.abs(np.diff(rr)) > 50)*1) / len(rr)]
    return results

In [18]:
def ecg_HRV_df(ecg_signals_1,info,name):
    results = {}
    results = pd.DataFrame(results)
    results =timedomain(ecg_signals_1,info,name,results)
    results =timedomain_HRV(ecg_signals_1,info,name,results)
    results =timedomain_pic(ecg_signals_1,info,name,results)
    return results

In [19]:
def ecg_HRV(ecg_signal,n_0,n_1):
    df_HTV =pd.DataFrame()
    #df = pd.DataFrame(ecg_signal[0])
    df = ecg_signal
    if n_1 - n_0 != 1:
        for j in range(n_0,n_1):

            ecg_signals_1, info  = nk.ecg_process(df[j], sampling_rate=500)
            HTV = ecg_HRV_df(ecg_signals_1,info,j)
            df_HTV[HTV.columns] = HTV
    elif n_1 - n_0 == 1:
        ecg_signals_1, info  = nk.ecg_process(df, sampling_rate=500)
        HTV = ecg_HRV_df(ecg_signals_1,info,n_0)
        df_HTV[HTV.columns] = HTV
    return df_HTV

In [20]:
def stardant_N(X_columns,fix_displacement_min, fix_displacement):
    for i in fix_displacement_min.columns:   
        X_columns[i] = X_columns[i] - fix_displacement_min[i]["min"]
    for i in fix_displacement.columns:   
        X_columns[i] = (X_columns[i] - fix_displacement[i]["mean"])/(fix_displacement[i]["std"])
    return X_columns 

## 2 


In [32]:
name_model = {
    "r":["LGBMClassifier.joblib","LinearSVC.joblib","lstm_model_r.onnx"],
    "d":["LGBMClassifier.joblib","ExtraTreesClassifier.joblib","HistGradientBoostingClassifier.joblib","lstm_model_d.onnx"]
}



In [ ]:
"""
led: Количиство отведений (12_led,6_led,12_led)
name_led: Имя отведения ((c 1 по 12), (6 - 3 стандартные + 3 усиленные,12 - грудные),(12 - для всех))
name_model_1: Имя модели (к примеру "LGBMClassifier.joblib")
name_d: Прогнозирование ритма или диагноза ("r","d" )
lan: Язык вывода ("ru","en")
HRV: Показатели вариабельности
path_def: Общий путь для папки в которой хранятся рабочие
"""

In [51]:
def path_s(led,name_led,name_model,name_d,lan,HRV,path_def) :
    path_n = led + name_model.split(".")[0]
    path_end = str(name_led-int(led.split("_")[0]))+"_"+str(name_led)+"_led_"+name_d + "."+name_model.split(".")[1]
        
    if name_d == "d":
        uniques = pd.read_csv( path_def +"diagnostic_2/uniques_diagnostic.csv", index_col=0)
        list_corr = pd.read_csv(path_def+"diagnostic_2/list_corr_diagnostic.csv", index_col=0)
        path = path_def + "diagnostic_2/" + path_n+path_end
        path_2 = path_def + "diagnostic_2/"+ path_n + "."+name_model.split(".")[1]
    if name_d == "r":
        uniques = pd.read_csv( path_def +"rhythm_2/uniques_rhythm.csv", index_col=0)
        list_corr = pd.read_csv(path_def+"rhythm_2/list_corr_rhythm.csv", index_col=0)
        path = path_def + "rhythm_2/" + path_n+path_end
        path_2 = path_def + "rhythm_2/"+ path_n + "."+name_model.split(".")[1]
    if "."+name_model.split(".")[1] == ".joblib":
            model = load(path)
            y_predict = model.predict(HRV)
            return uniques[lan][y_predict[0]]
    elif ("."+name_model.split(".")[1] == ".onnx")and led == "12_led/":
        #keras.models.load_model(path_2)
        model = ort.InferenceSession(path_2)
        #model.predict(HRV[list(list_corr["0"])[:-1]])
        H = HRV[list(list_corr["0"])[:-1]].to_numpy().astype(np.float32)
        y_predict = model.run(["output_names"], {'x': H})[0] 
        y_predict = torch.from_numpy(y_predict)
        max_in_ = pd.DataFrame(y_predict[0]).idxmax()[0]
        #закоментировать pred_namber если не нужен множественный вывод
        pred_namber = pd.DataFrame(y_predict[0])*100
        pred_namber[lan] = uniques[lan]
        pred_namber = pred_namber[pred_namber[0]>50]
        return uniques[lan][max_in_],pred_namber

In [ ]:
"""
ECG: Экг сигнал
age: Пол 
sex: Возраст
count_led: Количиство отведений (1,6,12)
name_led: Имя отведения ((c 1 по 12), (6 - 3 стандартные + 3 усиленные,12 - грудные),(12 - для всех))
name_model_1: Имя модели (к примеру "LGBMClassifier.joblib")
name_d: Прогнозирование ритма или диагноза ("r","d" )
lan: Язык вывода ("ru","en")
"""

In [23]:
class func_ecg_detect():
    def __init__(self,ECG,age,sex):
        self.path_def = "/kaggle/input/new-hrw-d/"
        self.ECG = ECG    
        self.age = age
        self.sex = sex
        self.fix_displacement = pd.read_csv(str(self.path_def + "fix_displacement.csv"), index_col=0)
        self.fix_displacement_min = pd.read_csv(str(self.path_def + "fix_displacement_min.csv"), index_col=0)

        
    def detect_led(self,count_led,name_led,name_model_1,name_d,lan):

        self.HRV_2 = ecg_HRV(self.ECG,name_led-count_led,name_led)
        self.HRV_2["age"] = self.age
        self.HRV_2["sex"] = self.sex
        self.HRV_2 = stardant_N(self.HRV_2,self.fix_displacement_min[self.HRV_2.columns], self.fix_displacement[self.HRV_2.columns])

        for i in name_model[name_diagnostic] :
            if name_model_1 in i:
                model = i
        path_2 = path_s(str(count_led)+"_led/",name_led,model,name_d,lan,self.HRV_2,self.path_def)

        return path_2
    
    

# Загрузка сигнала

In [24]:
import os
!pip install wget
import wget
import wfdb

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=09a9305515220046ee465bf93c1717b095a3f4ddd7fabeb8f4e3351767e5274c
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [25]:
d_measurements = pd.read_csv("/kaggle/input/electrocardiogram-database-for-arrhythmia-study/d_measurements.csv", index_col=0)

In [26]:
def load_signals_mimic(df):
    data = [wfdb.rdsamp(str(df.name_file[link]), pn_dir='mimic-iv-ecg/1.0/'+ df.path[link] + "/") for link in df.index]
    data = np.array([signal for signal, meta in data])
    return data

In [27]:
d_measurements[11110:11111]

,report,report_1,study_id,report_0,path,name_file
11351708,1AVB,sinus bradycardia with borderline 1st degree a...,45221309.0,*** consider acute st elevation mi ***,files/p1135/p11351708/s45221309,45221309


In [28]:
ecg_signal = load_signals_mimic(d_measurements[11110:11111])

## Проверка

In [45]:
d_measurements[11110:11111]

,report,report_1,study_id,report_0,path,name_file
11351708,1AVB,sinus bradycardia with borderline 1st degree a...,45221309.0,*** consider acute st elevation mi ***,files/p1135/p11351708/s45221309,45221309


In [52]:
Age, Sex = 59,1
# count_led количиство отведений
count_led = 12 
# name_otvedenie от 1 до 12 при диагностике 1 отведения
# 6 - при диагностике отведения первых 6 отведений,12 - при диагностике с 6 по 12 (грудные) 
# 12 -  при диагностике всех
name_otvedenie = 12
# диагноз - d, ритм - r
name_diagnostic = "d" 
name_model_1 = name_model[name_diagnostic][-1]#"HistGradientBoostingClassifier.joblib"
ecg_n_1 = pd.DataFrame(ecg_signal[0])

h = func_ecg_detect(ecg_n_1,Age, Sex)
h.detect_led(count_led,name_otvedenie,name_model_1,name_diagnostic,"ru")

('Неспецифические изменения ST',
            0                                        ru
 0  99.948341                            Нормальная ЭКГ
 4  68.439125                   Первая степень по блоку
 6  99.982574              Неспецифические изменения ST
 8  63.178505  Неполная блокада правой ножки пучка Гиса)

In [50]:
name_diagnostic = "d"
name_model_1 = name_model[name_diagnostic][1]
h = func_ecg_detect(ecg_n_1,Age, Sex)
h.detect_led(count_led,name_otvedenie,name_model_1,name_diagnostic,"ru")

'Первая степень по блоку'

In [60]:
Age, Sex = 59,1
count_led = 1 

name_diagnostic = "d" 
name_model_1 = name_model[name_diagnostic][0]
for i in range(0,12):
    name_otvedenie = i+1
    ecg_n_1 = pd.DataFrame(ecg_signal[0])[i]
    h = func_ecg_detect(ecg_n_1,Age, Sex)
    pred = h.detect_led(count_led,name_otvedenie,name_model_1,name_diagnostic,"ru")
    print(pred)

Неполная блокада правой ножки пучка Гиса
Нормальная ЭКГ
Нормальная ЭКГ
Нормальная ЭКГ
Нормальная ЭКГ
Нижний инфаркт миокарда
Нормальная ЭКГ
Нормальная ЭКГ
Нормальная ЭКГ
Нормальная ЭКГ
Неполная блокада правой ножки пучка Гиса
Неполная блокада правой ножки пучка Гиса


In [61]:
count_led = 6 
ecg_n_12 = pd.DataFrame(ecg_signal[0])[[6,7,8,9,10,11]]
ecg_n_6 = pd.DataFrame(ecg_signal[0])[[0,1,2,3,4,5]]
name_diagnostic = "d" 
name_model_1 = name_model[name_diagnostic][0]

h = func_ecg_detect(ecg_n_6,Age, Sex)
print(h.detect_led(count_led,6,name_model_1,name_diagnostic,"ru"))

h = func_ecg_detect(ecg_n_12,Age, Sex)
print(h.detect_led(count_led,12,name_model_1,name_diagnostic,"ru"))

Первая степень по блоку
Нормальная ЭКГ


In [ ]:
Age, Sex = 59,1
count_led = 1 
name_otvedenie = 4
ecg_n_1 = pd.DataFrame(ecg_signal[0])[4-1] # нумерация с 0 у индексов

name_diagnostic = "d" 
name_model_1 = name_model[name_diagnostic][1]

h = func_ecg_detect(ecg_n_1,Age, Sex)
h.detect_led(count_led,name_otvedenie,name_model_1,name_diagnostic,"ru")